# Gides markdown generator
Adopted from Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `guides.py`. Run either from the `markdown_generator` folder after replacing `guides.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: title, author, and paper_url, with a header at the top.

- `author` can be blank, but the other must have vlaues.

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [83]:
!cat guides.tsv

## Import pandas

We are using the very handy pandas library for dataframes.

In [84]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [85]:
guides = pd.read_csv("guides.tsv", sep="\t", header=0)
guides


,id,title0,title1,excerpt,url_slug,paper_url
0,1,Code for America,Brigade Organizer's Playbook,This organizer's playbook will help hackathon ...,code-for-America-playbook,http://eipapa.github.io/hackathon-planning-kit...
1,2,Major League Hacking (MLH),The MLH Hackathon Organizer Guide,Major League Hacking (MLH) is the official sut...,mlh,https://guide.mlh.io
2,3,Open Data Day DC by Joshua Tauberer,How to run a successful hackathon (Open Data D...,This guide is created out of the author's expe...,open-data-day-dc,https://hackathon.guide
3,4,Chicago Civic Tech,@Civicwhitaker Anthology,This is the comprehensive collection of civic ...,civicwhitaker-anthology,http://eipapa.github.io/hackathon-planning-kit...
4,5,Hackathon Playbook,NaN,This playbook outlines the guidlines for desig...,hackathon-playbook-external,http://eipapa.github.io/hackathon-planning-kit...
5,6,Health Hackathon Handbook,MIT Hacking Medicine,"This guide provides the who, what, when, where...",health-hackathon-handbook,http://eipapa.github.io/hackathon-planning-kit...
6,7,The complete guide to organizing a successful ...,Hackerearth,This guide constructed based on the expertise ...,hackerearth,http://eipapa.github.io/hackathon-planning-kit...


In [86]:
for row, item in guides.iterrows():
    print(item.excerpt)

This organizer's playbook will help hackathon organizers to get started. This guide was build from Code for America's research about how to build sustainable, civic hacking groups and bring 21st century government practices and products to their cities. This guide contains various check-lists essential for organizing, building, and infrastructure development.
Major League Hacking (MLH) is the official sutdent hackathon league in the US which was launched in 2013. Each year, MLH powers over 200 weekend-long invention competitions that inspire innovation, cultivate communities and teach computer science skills to more than 65,000 students around the world. MLH organizer's guide is an essential resource for student hackathon organizers as it contains lots of lessons that MLH and the hackathon community around the world have learnt from organizing the hackathons. This guide organizes activities that organizers need to perform by timeline and also includes other useful resources such as exa

## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [87]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [88]:
import os
for row, item in guides.iterrows():
    
    md_filename = str(item.id) + "-" + item.url_slug + ".md"
    html_filename = str(item.id) + "-" + item.url_slug
    
    ## YAML variables
    
    md = "---\ntitle0: \"" + item.title0 + "\""
    if len(str(item.title1)) > 5:
        md += "\ntitle1: \"" + item.title1 + "\""
    else:
        md += "\ntitle1: \"" + "\""
    md += "\ncollection: guides"
#     md += """\npermalink: /guide/""" + html_filename
#     md += "\nvenue: '" + html_escape(item.venu) + "'"
    md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    md +="\npaper_url: '" + item.paper_url + "'"
    md += "\n---"
    
    ## Markdown description for individual page
        
#     if len(str(item.paper_url)) > 5:
#         md += "\n[<span style=\"color: #c41e3a\">Download PDF here.</span>](" + item.paper_url + ")\n"
    md += "\n'" + html_escape(item.excerpt) + "\'"
    md += "\n[Download PDF here.](" + item.paper_url + ")\n"
#     if len(str(item.excerpt)) > 5:
#         md += "\n**Abstract**:" + html_escape(item.excerpt) + "\n"    
#     md += "\nAbstract: " + html_escape(item.description) + "\n"
#     md += "\n**Recommended citation**: " + item.citation
    md_filename = os.path.basename(md_filename)
    with open("../_guides/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [89]:
!ls ../_guides/

1-code-for-America-playbook.md   5-hackathon-playbook-external.md
2-mlh.md                         6-health-hackathon-handbook.md
3-open-data-day-dc.md            7-hackerearth.md
4-civicwhitaker-anthology.md


In [ ]:
!cat ../_guides/2009-10-01-paper-title-number-1.md